In [2]:
import csv

In [3]:
import numpy as np

In [4]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from matplotlib import pyplot as plt

In [6]:
packet_data=[]

In [7]:
with open ('vdg1nb.csv') as csvfile:
    data_csv = csv.reader(csvfile, delimiter=',')
    for row in data_csv:

        packet_data.append(row) 
  

In [8]:
packet_data=np.array(packet_data)

In [9]:
col=len(row)

In [10]:
row=len(packet_data)

In [11]:
Data=np.zeros((92349,7))

In [12]:
labels=np.zeros((92349,1))

In [13]:
data_dim=Data.shape

In [14]:
n_row=data_dim[0]

In [15]:
n_col=data_dim[1] 

In [16]:
#parameters

In [17]:
n_iterations=100

In [18]:
init_learning_rate=0.01

In [19]:
eta_null=init_learning_rate

In [20]:
map_size_x=100

In [21]:
map_size_y=100

In [22]:
n_weights=map_size_x*map_size_y*n_col

In [23]:
network_dimensions = np.array([5, 5])

In [24]:
sigma_null = max(network_dimensions[0], network_dimensions[1]) / 2

In [25]:
time_constant=n_iterations/np.log(sigma_null)

In [26]:
#Normalize for data in 0,1

In [27]:
Data_normed=Data/Data.max(axis=0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [28]:
#Initialization of weights

In [29]:
W=np.random.random_sample((map_size_x*map_size_y,n_col))

In [30]:
init_W=W

In [31]:
sess=tf.Session()

In [33]:
#Function gets the distance between single input sample and all the cells in the feature map to find winner weight.
def BMU(x,w):
    x_t=tf.convert_to_tensor(x)
    x_t=tf.to_float(x_t)
    x_tt=tf.expand_dims(x_t,0)
    w_t=tf.convert_to_tensor(w)
    w_t=tf.to_float(w_t)
    w_tt=tf.expand_dims(w_t,1)
    subb=tf.subtract(x_tt,w_tt)
    sq=tf.square(subb)
    distance=tf.reduce_sum(sq,2)#this shows the distance of every point from the rest of the points
    distance=tf.sqrt(distance)
    dist=sess.run(distance)
    #find the minium distance and its index
    ind = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
    return [ind, dist[ind]]
        
def find_index(ncol, index):
    IND=index[0]
    r=int(np.floor(IND/ncol)); #row of the minimum value in the feature map, if we assume that the vectorization was row-wise
    c=int(np.mod(IND,ncol));
    return [r,c]
       

def neighborhood_decrease(sigma_null, t, time_constant):
    Sigma=sigma_null * np.exp(-t / time_constant)
    return Sigma

def learning_rate_decrease(eta_null, t, n_iterations):
    eta=eta_null * np.exp(-t / n_iterations)
    return eta

def topological_neighborhood(distance_S, Sigma):
    return np.exp(-distance_S / (2* (Sigma**2)))

In [34]:
#Main part
for t in range (n_iterations): #for each iteration
    print('iteration: ',t)
    for j in range (n_row):#for each sample of the dataset
        print('sample: ',j)
        s=Data_normed[j]#each sample has 30 features (columns)
        [index,min_distance]=BMU(Data_normed[j],W)
        [ro,co]=find_index(map_size_y, index)#we can find the 2Dindex of the minimum value
        # decrease the SOM parameters to modify the weights 
        Sigma = neighborhood_decrease(sigma_null, t, time_constant)
        Eta = learning_rate_decrease(eta_null, t, n_iterations)    
        # update the weights
        r=Sigma
        for k1 in range (map_size_x):
           for k2 in range (map_size_y):
               if abs(k1-ro)<=r and abs(k2-co)<=r:##finding the neighbors for the winner
                       indd=int(map_size_y*(k1-1)+k2)#the vectorized index of the weight
                       distance_S=np.sqrt(np.sum((np.array([k1,k2])-np.array([ro,co]))**2))
                       T=topological_neighborhood(distance_S, Sigma)
                       new_w = W[indd] + (Eta * T * (Data_normed[j]- W[indd]))
                       W[indd]=new_w

iteration:  0
sample:  0
sample:  1
sample:  2
sample:  3
sample:  4
sample:  5
sample:  6
sample:  7
sample:  8
sample:  9
sample:  10
sample:  11
sample:  12
sample:  13
sample:  14
sample:  15
sample:  16
sample:  17
sample:  18
sample:  19
sample:  20
sample:  21
sample:  22
sample:  23
sample:  24
sample:  25
sample:  26
sample:  27
sample:  28
sample:  29
sample:  30
sample:  31
sample:  32
sample:  33
sample:  34
sample:  35
sample:  36
sample:  37
sample:  38
sample:  39
sample:  40
sample:  41
sample:  42
sample:  43
sample:  44
sample:  45
sample:  46
sample:  47
sample:  48
sample:  49
sample:  50
sample:  51
sample:  52
sample:  53
sample:  54
sample:  55
sample:  56
sample:  57
sample:  58
sample:  59
sample:  60
sample:  61
sample:  62
sample:  63
sample:  64
sample:  65
sample:  66
sample:  67
sample:  68
sample:  69
sample:  70
sample:  71
sample:  72
sample:  73
sample:  74
sample:  75
sample:  76
sample:  77
sample:  78
sample:  79
sample:  80
sample:  81
sample:  82


sample:  638
sample:  639
sample:  640
sample:  641
sample:  642
sample:  643
sample:  644
sample:  645
sample:  646
sample:  647
sample:  648
sample:  649
sample:  650
sample:  651
sample:  652
sample:  653
sample:  654
sample:  655
sample:  656
sample:  657
sample:  658
sample:  659
sample:  660
sample:  661
sample:  662
sample:  663
sample:  664
sample:  665
sample:  666
sample:  667
sample:  668
sample:  669
sample:  670
sample:  671
sample:  672
sample:  673
sample:  674
sample:  675
sample:  676
sample:  677
sample:  678
sample:  679
sample:  680
sample:  681
sample:  682
sample:  683
sample:  684
sample:  685
sample:  686
sample:  687
sample:  688
sample:  689
sample:  690
sample:  691
sample:  692
sample:  693
sample:  694
sample:  695
sample:  696
sample:  697
sample:  698
sample:  699
sample:  700
sample:  701
sample:  702
sample:  703
sample:  704
sample:  705
sample:  706
sample:  707
sample:  708
sample:  709
sample:  710
sample:  711
sample:  712
sample:  713
sample:  714

KeyboardInterrupt: 

In [ ]:
#Plot the SOM  
Weight_mat=np.zeros((map_size_x,map_size_y,30))                       
for k3 in range (len(W)):
    ro1=int(np.floor(k3/map_size_y)); #row of the minimum value in the feature map, if we assume that the vectorization was row-wise
    col1=int(np.mod(k3,map_size_y));
    Weight_mat[ro1,col1]=W[k3]
Mn=tf.reduce_mean(Weight_mat,axis=2)
Meann=sess.run(Mn)
plt.imshow(Meann, cmap='hot')  
plt.show() 